In [32]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from helpers import *
from matplotlib import pyplot as plt
import numpy as np
from experiment_mnist import *
from experiment_constant import *
from experiment_random import *
from scipy.special import expit
import pandas as pd
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
p = 1e-2
exp = RandomExperiment([100, 21, 22, 23, 1], [p, 0, 0], 1, 'sigmoid')
model = exp.model_no_dropout

### Testing theorem 1:
$$\sum\limits_{i,j}\frac{\partial y_k}{\partial w_{ij}}w_{ij}=\sum\limits_{j}\frac{\partial y_k}{\partial x_j}x_j
    = -\frac{1}{p}(\mathbb{E}_\xi\Delta_L^k+O(p^2))$$

In [34]:
some_x = -np.random.rand(100) * 100

In [35]:
def val_for_some_x(tensor):
    """ Evaluate tensor for some fixed input """
    return sess.run(tensor, feed_dict = {model.layers[0].input.name: [some_x]})

In [36]:
def grad_and_sum(output, variable):
    """ Calculate sum(doutput/dvariable * variable) """
    dydvar = tf.gradients(model.layers[-1].output, variable)[0]
    mult = tf.multiply(dydvar, variable)
    return tf.reduce_sum(mult)

In [37]:
def calc_and_print(layer = 0):
    """ Calculate and print quantities from Th1 for a layer """
    dydx = grad_and_sum(model.layers[-1].output, model.layers[layer].input)
    dydw = grad_and_sum(model.layers[-1].output, model.layers[layer].weights[0])
    return(val_for_some_x(dydx), val_for_some_x(dydw))

In [38]:
model.predict(some_x.reshape(1, -1))
for layer in range(len(model.layers)):
    print(layer, calc_and_print(layer))

0 (6.1103376e-05, 6.110314e-05)
1 (-0.0016422537, -0.0016422544)
2 (-0.017006662, -0.017006658)
3 (0.14561379, 0.14561379)


In [45]:
-1. / p * np.mean(exp.get_error(some_x, repetitions = 1000000))

-0.001690919089014642

In [40]:
exp.get_exact_error_v3(some_x), exp.get_exact_error_v3_better(some_x)

(array([[1.64225295e-05]]), array([[1.7153025e-05]], dtype=float32))